# Scientific Python
## Central European University, Fall 2023/2024

Instructor: Tiago Peixoto (standing in for Márton Posfai)

Emails: <peixotot@ceu.edu>

# Networks

## Brief introduction

<b>Networks (or graphs)</b> are mathematical objects composed of two sets $N$ and $E$, of vertices (a.k.a. nodes) and edges (a.k.a. links). Each edge in $E$ is pair of nodes in $V$.

<img src="social_networks2.gif" alt="a Network" style="width:304px;height:228px;">


Basic types of graphs:
* **Undirected** or **directed** networks. The friendship network in Facebook is an example of an undirected graph, twitter is a directed graph.
* **Simple graphs** or **multigraphs**: In simple graphs only one edge is allowed between two nodes. In multigraphs this restriction is lifted.
* **Unweighted** or **weighted**: the friendship network in Facebook is unweighted, the airport network (nodes: airports, links: number of airplanes flying between two airports) is weighted. 

Jargon:
* Node $j$ is a **neighbor** of node $i$ if the edge $(i,j)$ exists. The **neighborhood** of $i$ is the set of all $i$'s neighbors.
* The **degree** of $i$ is the number of its neighbors. The **degree distribution** $P(k)$ is the probability that a randomly chosen node in the network has degree $k$.
* **Hubs** are nodes with very high degree. In many networks (the Internet, social networks, etc.) they are rare but much more common than you may expect.
* **Sparse networks**: Most pairs of nodes do not have a link. Real-world networks are of this kind.
* A **path** between two nodes, say $i$ and $j$, is the sequence of nodes that you need to traverse in order to get from $i$ to $j$ (and from $j$ to $i$ in case of undirected graphs).
* The **distance** between nodes $i$ and $j$ is the length of the *shortest path* between them. The **diameter** of a network is the *longest* shortest path.
* The local **clustering coefficient** $c_i$ of node $i$ is the fraction of neighbors of $i$ that are linked: 

  $$c_i = \frac{T_i}{\binom{k_i}{2}} = \frac{2T_i}{k_i(k_i-1)}$$

  where $T_i$ is the number of triangles to which node $i$ belongs (i.e. the number of connected neighbors). In other words, it quantifies how many of $i$'s 'friends' also 'friends' with each other.

Specific graphs:
* **Complete graphs**: every link exists $\binom{n}{2}$ links for $n$ nodes.
* **Lattices**
* **Erdős-Rényi** random graphs: Every possible edge exists with probability $p$
* Many more



### Working with networks in Python

There are several network packages for Python:

* NetworkX: https://networkx.github.io/
* igraph: https://igraph.org/
* graph-tool: https://graph-tool.skewed.de/

In this course whe will use `graph-tool`. Why? See a performance comparison here: https://graph-tool.skewed.de/performance

NetworkX is implemented in pure Python. Instead, `graph-tool` follows the Numpy philosophy, and implements the core data structure and algorithms in a lower-level language, C++ with [templates](https://en.wikipedia.org/wiki/Template_(C%2B%2B)). This means a performance improvement of up to 200x.

**But there is no free lunch!** Implementing a library in C++ means that it requires the C++ infrastructure and environmente to be compiled and installed in different architectures. C++ is not as portable as (pure) Python, so a program compiled for GNU/Linux does not work in MacOS, etc.

If using MacOS or GNU/Linux, the easiest way to do this is by using [anaconda](https://www.anaconda.com/):

```
conda create --name gt -c conda-forge graph-tool
conda activate gt
```

Further instructions to install graph-tool in various systems are available here: https://git.skewed.de/count0/graph-tool/wikis/installation-instructions

``graph-tool`` has **lots** of documentation that you should definitely read: https://graph-tool.skewed.de/static/doc

If you open this notebook on https://colab.research.google.com you will be able to install graph-tool by running the following cell (run it only once!).

In [ ]:
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-cairo python3-matplotlib

### Creating a Graph

In [ ]:
import numpy as np
from graph_tool.all import *

g = Graph()            # create an empty graph with no vertices and no edges. 
print(g)

# By default graphs are directed. If you wish undirected graphs, you need to pass the option: directed=False

g = Graph(directed=False)
print(g)

### Vertices
Adding vertices (nodes) are done with the ``Graph.add_vertex()`` method.

In [ ]:
v = g.add_vertex()    # add a single vertex, and returns the vertex object
print(v)

In [ ]:
vs = g.add_vertex(10)    # you can add many vertices at once, and an iterator to the vertices added is returned
print(list(vs))
print(g.num_vertices())

Vertices are always indexed from `0` to `N-1` where `N` is the total number of vertices.

We can always obtain the vertex object directly from the index:

In [ ]:
v = g.vertex(5)
v

### Edges

We can add edges using ``Graph.add_edge()``.

In [ ]:
v1 = g.vertex(0)
v2 = g.vertex(1)

e = g.add_edge(v1, v2)

print(repr(e))

# we can also use the vertex index directly:

e2 = g.add_edge(0, 2)

print(repr(e2))

print(g.num_edges())

We can add many edges at once using `Graph.add_edge_list()`:

In [ ]:
g = Graph(directed=False)
g.add_edge_list([(0, 1), (2, 3), (1, 0), (3, 4)])  # non-existing vertices are automatically added!

print(g)

In [ ]:
# We can lookup the existence of edges with the ``Graph.edge()`` method:

e = g.edge(2, 3)
print(e)

e = g.edge(2, 4)
print(e)


# We can query the source and target of an edge:

e = g.edge(2, 3)

print(e.source(), e.target())

# We can also convert an edge to a tuple:

u, v = e

print(u, v)
print(type(u))

### Iterating over vertices and edges

In [ ]:
for v in g.vertices():
    print(v)
    
for e in g.edges():
    print(e)

for u, v in g.edges():
    print(u, v)
    
for v in g.vertices():
    print(f"Edges incident on {v}: ", end="")
    for e in v.out_edges():
        print(e, end=" ")
    print()
    
for v in g.vertices():
    print(f"The degree of node {v} is {v.out_degree()} and its neighbors are: ", end="")
    for u in v.out_neighbors():
        print(u, end=" ")
    print()

### Directed graphs

In [ ]:
g = Graph()
g.add_edge_list([(0, 1), (2, 3), (1, 0), (3, 4)])

print(g)

for v in g.vertices():
    print(f"Outgoing edges from {v}: ", end="")
    for e in v.out_edges():
        print(e, end=" ")
    print()
    
    print(f"Incoming edges to {v}: ", end="")
    for e in v.in_edges():
        print(e, end=" ")        
    print()
    
for v in g.vertices():
    print(f"The out-degree of node {v} is {v.out_degree()} and its out-neighbors are: ", end="")
    for u in v.out_neighbors():
        print(u, end=" ")
    print()

    print(f"The in-degree of node {v} is {v.in_degree()} and its in-neighbors are: ", end="")
    for u in v.in_neighbors():
        print(u, end=" ")
    print()



### Drawing Graphs

``graph-tool`` has sophisticated routines for drawing graphs.

In [ ]:
graph_draw(g, vertex_text=g.vertex_index, output_size=(200, 200));

### Exercise 1

Create and draw the following graph:

<img src="https://upload.wikimedia.org/wikipedia/commons/9/96/K%C3%B6nigsberg_graph.svg"/>


<details><summary><u>Solution.</u></summary>
<p>

```python
g = Graph(directed=False)
for u, v in [(0,1), (0,1), (1,2), (2,0), (2, 3), (3, 0), (3, 0)]:
    g.add_edge(u, v)
graph_draw(g, output_size=(200,200));
```
</p>
</details>
    

### Exercise 2

Using the graph created above:

* Count the number of edges
* Count the number of nodes
* Calculate the average degree per node
* Calculate the maximum and minimum number of neighbors
* Calculate the number of nodes with degree = 3

<details><summary><u>Solution.</u></summary>
<p>
    
```python
print("Number of edges:", g.num_edges())
print("Number of nodes:", g.num_vertices())
print("Average degree:", 2 * g.num_edges() / g.num_vertices())
print("Average degree:", np.mean([v.out_degree() for v in g.vertices()]))
print("Maximum degree:", max([v.out_degree() for v in g.vertices()]))
print("Minimum degree:", min([v.out_degree() for v in g.vertices()]))
print("Number of nodes with degree 3:", len([v for v in g.vertices() if v.out_degree() == 3]))
```
</p>
</details>

## Property maps

In ``graph-tool`` we can attribute nodes and edges with arbitrary properties using property maps.

Property maps can be of the following types:

    
|Type name                     | Alias                                   |
|------------------------------|-----------------------------------------|
|``bool``                      |    ``uint8_t``                          |
|``int16_t``                   |    ``short``                            |
|    ``int32_t``               |    ``int``                              |
|    ``int64_t``               |    ``long``, ``long long``              |
|    ``double``                |    ``float``                            |
|    ``long double``           |                                         | 
|    ``string``                |                                         |
|    ``vector<bool>``          |    ``vector<uint8_t>``                  |
|    ``vector<int16_t>``       |    ``vector<short>``                    |
|    ``vector<int32_t>``       |    ``vector<int>``                      |
|    ``vector<int64_t>``       | ``vector<long>``, ``vector<long long>`` |
|    ``vector<double>``        |    ``vector<float>``                    |
|    ``vector<long double>``   |                                         |
|    ``vector<string>``        |                                         |
|    ``python::object``        |   ``object``                            |



In [ ]:
g = Graph()
g.add_edge_list([(0, 1), (2, 3), (1, 0), (3, 4)])

vsize = g.new_vp("int")      # new vertex property map of type int
eweight = g.new_ep("double") # new edge property map of type double

for v in g.vertices():
    vsize[v] = 10
    
for e in g.edges():
    eweight[e] = 3.2
    
# We can also access the values of property maps as numpy arrays:

print(vsize.a)

vsize.a = [3, 10, 5, 1, 15]
vsize.a *= 4

vsize.a[4] = 3
vsize[g.vertex(4)] = 3

print(g.vertex_index[g.vertex(4)])
print(g.edge_index[g.edge(2,3)])


eweight.a = [3.2] * 2 + [1., 2.]
    
# property maps can be used with many functions, e.g. graph_draw()

graph_draw(g, vertex_size=vsize, vertex_fill_color=vsize, edge_pen_width=eweight, output_size=(300, 300));


In [ ]:
# shortest paths

g = lattice([25, 25])

vertices, edges = shortest_path(g, g.vertex(0), g.vertex(g.num_vertices() - 1))

ecolor = g.new_ep("string", val="black")
vcolor = g.new_vp("string", val="black")

for v in vertices:
    vcolor[v] = "red"
for e in edges:
    ecolor[e] = "red"

pos = sfdp_layout(g, multilevel=True)

graph_draw(g, pos=pos, vertex_fill_color=vcolor, edge_color=ecolor, output_size=(300, 300));


In [ ]:
# now we use random weights

import numpy.random

eweights = g.new_ep("double")
eweights.a = numpy.random.random(len(eweights.a))
print(eweights.a)

vertices, edges = shortest_path(g, g.vertex(0), g.vertex(g.num_vertices() - 1), weights=eweights)

ecolor = g.new_ep("string", val="black")
vcolor = g.new_vp("string", val="black")

for v in vertices:
    vcolor[v] = "red"
for e in edges:
    ecolor[e] = "red"

ewidth = eweights.copy()
ewidth.a = (1-ewidth.a) * 1.5
    
graph_draw(g, pos=pos, vertex_fill_color=vcolor, edge_color=ecolor, edge_pen_width=ewidth, output_size=(400,400));

#### Exercise 3

Consider the weighted undirected graph corresponding to the following list of edges:

    (a, b) weight = 0.6
    (a, c) weight = 0.2
    (c, d) weight = 0.1
    (c, e) weight = 0.7
    (c, f) weight = 0.9
    (a, d) weight = 0.3

* Create a graph with the edges above and two property maps, vlabel and eweight, with the vertex labels and edge weights, respectively.
* Draw the graph, using the edge weight as edge width.
* Compute the shortest path from 'b' to 'e' and draw it.
* Change the edge weights so that the shortest path goes through 'd'.

<details><summary><u>Solution.</u></summary>
<p>

```python
vs = ['a', 'b', 'c', 'd', 'e', 'f']
elist = [('a', 'b', .6),
         ('a', 'c', .2),
         ('c', 'd', .1),
         ('c', 'e', .7),
         ('c', 'f', .9),
         ('a', 'd', .3)]

# doing it by 'hand'
g = Graph(directed=False)
vlabel = g.new_vp("string")
eweight = g.new_ep("double")

vmap = {}
for i, v in enumerate(g.add_vertex(len(vs))):
    vlabel[v] = vs[i]
    vmap[vs[i]] = v

for s, t, w in elist:
    e = g.add_edge(vmap[s], vmap[t])
    eweight[e] = w
    
graph_draw(g, vertex_text=vlabel, vertex_font_size=20, edge_pen_width=prop_to_size(eweight, 1, 10),
           output_size=(300,300));
    
# Doing it using Graph.add_edge_list()
g = Graph(directed=False)
eweight = g.new_ep("double")
vlabel = g.add_edge_list(elist, hashed=True, eprops=[eweight])
pos = graph_draw(g, vertex_text=vlabel, vertex_font_size=20, edge_pen_width=prop_to_size(eweight, 1, 10),
                 output_size=(300,300))    

# Shortest paths
vmap = {}
for v in g.vertices():
    vmap[vlabel[v]] = v
    
vs, es = shortest_path(g, vmap["b"], vmap["e"], weights=eweight)
print(vs)

ecolor = g.new_ep("int")
for e in es:
    ecolor[e] = 1
    
graph_draw(g, pos=pos, vertex_text=vlabel, vertex_font_size=20, edge_pen_width=prop_to_size(eweight, 1, 10),
           edge_color=ecolor, output_size=(300,300));

# Make the shortest path go through "d"

e = g.edge(vmap["a"], vmap["c"])
eweight[e] = 10

vs, es = shortest_path(g, vmap["b"], vmap["e"], weights=eweight)

ecolor.a = 0
for e in es:
    ecolor[e] = 1
    
graph_draw(g, pos=pos, vertex_text=vlabel, vertex_font_size=20, edge_pen_width=prop_to_size(eweight, 1, 10),
           edge_color=ecolor, output_size=(300,300));    
    
```
</p>
</details>

### Exercise 4
Create a graph with 20 nodes, and randomly add 20 edges. Plot the resulting graph.

<details><summary><u>Solution.</u></summary>
<p>
```python
g = Graph(directed=False)
g.add_vertex(20)
for i in range(20):
    u = v = 0
    while u == v or g.edge(u, v) is not None:
        u = np.random.randint(20)
        v = np.random.randint(20)
    g.add_edge(u, v)
graph_draw(g, output_size=(200, 200));
```
</p>
</details>

### Random graphs

In [ ]:
# Let's generate an ER graph (as a special case of the configuration model)
g = random_graph(1000, lambda: numpy.random.poisson(3), directed=False)
graph_draw(g);

### Exercise 5
Plot the degree distribution (histogram) of the Erdős-Rényi graph generated as:
```python
g = random_graph(100000, lambda: numpy.random.poisson(5), directed=False)

```

<details><summary><u>Solution.</u></summary>
<p>

```python
g = random_graph(100000, lambda: numpy.random.poisson(5), directed=False)

#degrees = [v.out_degree() for v in g.vertices()]

d = g.degree_property_map("out")

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

plt.hist(d.a, bins=np.arange(0, d.a.max()+1), density=True);
x = np.linspace(0, 18, 1000)
plt.plot(x, scipy.stats.poisson.pmf(5, x))
```
</p>
</details>


### Exercise 6

Consider the following cell, where a random graph is generated, with a Poisson degree distribution with an average of 3. The function `label_components()` returns a property map `c` and a distribution of component sizes `s`.

* Print out the size of the largest component.
* Obtain a vertex property map with value 1 if the vertex belongs to the largest component, 0 otherwise.

<details><summary><u>Solution.</u></summary>
<p>

```python
print(s.max())
r = s.argmax()
lc = g.new_vp("bool", vals=c.a == r)
graph_draw(g, vertex_fill_color=lc, output_size=(300, 300))
```
</p>
</details>


In [ ]:
import numpy
g = random_graph(1000, lambda: numpy.random.poisson(3), directed=False)
c, s = label_components(g)
graph_draw(g, vertex_fill_color=c);

### Exercise 7
The cell below generates a Barabási-Abert network with N=2000 nodes and m=1 new-node degrees.

Plot the degree distribution of a BA network with N=100000 and m=2 in log-log scale. (Do not try to draw it; it would take very long.)
<details><summary><u>Solution.</u></summary>
<p>
    
```python
g = price_network(N=100000, m=2, directed=False) # Barabasi-Albert network
# degs = [v.out_degree() for v in g.vertices()]
degs = g.degree_property_map("out").a

fig = plt.figure()
plt.hist(degs, bins=np.logspace(1, np.log10(max(degs)), 40), log=True, density=True);
plt.xscale("log")
```
</p>
</details>

In [ ]:
g = price_network(N=2000, m=1, directed=False) # Barabasi-Albert network

## Removing nodes and edges


In [ ]:
g = Graph()
g.add_vertex(10)

# Vertices that remain are always in the range [0,N-1]!

v = g.vertex(5)
g.remove_vertex(v)

for v in g.vertices():
    print(v)
    
g.remove_vertex([2,6,1])  # many vertices can be removed at once
    
    
# However, property maps behave as expected:

g = Graph()
g.add_vertex(10)

value = g.new_vp("int", vals=[int(v) for v in g.vertices()])
print(value.a)

g.remove_vertex(g.vertex(5))
print(value.a)


# Edges are removed in a similar manner

g.add_edge(0, 3)
g.add_edge(2, 5)

for e in g.edges():
    print(e)

e = g.edge(0, 3)
g.remove_edge(e)

for e in g.edges():
    print(e)

### Exercise

* Create a graph from the edge list `ego250.list`.
* Plot the network
* Remove the node 250 (use `Graph.remove_vertex()`)
* Detect the connected components
* Keep only the largest components (delete the rest)
* Plot the degree distribution

<details><summary><u>Solution.</u></summary>
<p>

```python
import numpy as np    
    
g = Graph(directed=False)
edges = np.loadtxt("ego250.list")
labels = g.add_edge_list(edges, hashed=True)
print(g)
pos = graph_draw(g, vertex_text=labels);

v = find_vertex(g, labels, 250)
print(v)

g.remove_vertex(v)

graph_draw(g, pos, vertex_text=labels);

c, s = label_components(g)

c_max = s.argmax()

g.remove_vertex([v for v in g.vertices() if c[v] != c_max])

graph_draw(g, pos, vertex_text=labels, output_size=(300, 300));
```
</p>
</details>

### Exercise 6
* Create large Erdős-Rényi graphs with mean degrees in the range `linspace(0, 3, 40)`.
* Plot the size of the largest component versus mean degree $\left<k\right>$
* Calculate the variance of the size of the connected components <b>without the largest one</b>  and plot them against $\left<k\right>$

<details><summary><u>Solution.</u></summary>
<p>

```python
ks = np.linspace(0, 3, 40)
N = 10000
S = np.zeros(len(ks))
s_v = np.zeros(len(ks))

for i, k in enumerate(ks):
    g = random_graph(N, lambda: numpy.random.poisson(k), directed=False)
    c, s = label_components(g)
    S[i] = s.max() / N
    s = [s[i] for i in range(len(s)) if i != s.argmax()]
    s_v[i] = np.var(s)
    
plt.plot(ks, S)
plt.plot(ks, np.array(s_v) / 30)    
```
</p>
</details>


## Graph IO and internal property maps

In [ ]:
g = random_graph(10000, lambda: (3,3))
g.save("g.gt") # The gt file format is a binary format that is very efficient

u = load_graph("g.gt")
print(similarity(u, g))

# Other fine formats are also supported
g.save("g.xml")  # GraphML file format
g.save("g.gml")  # GML file format
g.save("g.dot")  # Dot file format

# Compression can be achieved by appending ".gz", ".bz2" or ".xz" to the file names:

g.save("g.gt.gz")
g.save("g.xml.bz2")
g.save("g.dot.xz")

u = load_graph("g.xml.bz2")
print(similarity(u, g))

In [ ]:
# If we want to store property maps with out graph, we need to make them internal

eweight = g.new_ep("double", vals=numpy.random.random(g.num_edges()))
vcolor = g.new_vp("int", vals=numpy.random.randint(0, 10, g.num_vertices()))

g.ep["eweight"] = eweight
g.vp["vcolor"] = vcolor

g.list_properties()


g.save("g.gt")

u = load_graph("g.gt")
u.list_properties()

eweight = g.ep["eweight"]
vcolor = g.vp["vcolor"]

print(similarity(g, u, g.ep["eweight"], u.ep["eweight"]))


In [ ]:
# Shortcuts for property maps. The following two statements are equivalent:

print(g.ep["eweight"].a)
print(g.ep.eweight.a)


## Graph filtering and graph views

In [ ]:
g = collection.data["polblogs"]
g.list_properties()
graph_draw(g, pos=g.vp.pos);

In [ ]:
c = label_largest_component(g, directed=False)
u = GraphView(g, vfilt=c)
graph_draw(u, u.vp.pos);

In [ ]:
print(u)

In [ ]:
import matplotlib.cm

vb, eb = betweenness(u)
graph_draw(u, pos=u.vp.pos, vertex_color=vb, vertex_fill_color=vb, vertex_size=prop_to_size(vb, 5, 20),
           vorder=vb, vcmap=matplotlib.cm.plasma);

In [ ]:
# Graph views can be composed.

u.list_properties()

In [ ]:
# let's plot only the networks of right wing blogs
w = GraphView(u, vfilt=lambda v: u.vp.value[v] == 1)   # the supplied lambda function evaluates to True 
                                                       # for vertices that are *kept* in the graph
    
graph_draw(w, w.vp.pos);

In [ ]:
# Let's look only at the connections between left and right wing
w = GraphView(u, efilt=lambda e: u.vp.value[e.source()] != u.vp.value[e.target()])
graph_draw(w, w.vp.pos);

In [ ]:
# We can transform graphs between directed and undirected (and vice-versa) using GraphView as well:

w = GraphView(u, directed=False)
graph_draw(w, w.vp.pos);

## Caveat: property map arrays and filtered graphs

In [ ]:
w = GraphView(u, vfilt=lambda v: u.vp.value[v] == 1)
print(w.num_edges())
eweight = w.new_ep("int")
print(len(eweight.a))  # The array view *always* corresponds to the unfiltered graph underneath!

# For filtered graphs, we need to use the .fa attribute (which returns a view into the array)

print(len(eweight.fa))

eweight.fa = 10
print(eweight.fa)
print(eweight.a)

### Exercise 7

* Using `GraphView`, write a function that gets the undirected version of a directed graph and filters out vertices of degree 3 or smaller, returning the result.

* Use the function above on the `polblogs` network.

* What happens if you run that function iteratively, multiple times?

In [ ]:
def kfilt(g):
    u = GraphView(g, directed=False)
    return GraphView(u, vfilt=lambda v: v.out_degree() > 3)
g = collection.data["polblogs"]
u = kfilt(g)
graph_draw(u, pos=u.vp.pos)

print(g.num_vertices())
u = g
for i in range(4):
    u = kfilt(u)
    print(u.num_vertices())
graph_draw(u, pos=u.vp.pos)

# Community detection and statistical inference

Graph-tool has extensive functionally to detect modules (or "communities") of nodes using principled statistical inference approaches. A detailed howto can be found here: https://graph-tool.skewed.de/static/doc/demos/inference/inference.html

In [ ]:
g = collection.data["polblogs"]
g = extract_largest_component(g, directed=False, prune=True)

graph_draw(g, pos=g.vp.pos, output_size=(300, 300));

In [ ]:
# Suppose we want to figure out the groups, without "looking" at the network or using the metadata.
# We can do this by fitting the stochastic block model (SBM):

state = minimize_blockmodel_dl(g, multilevel_mcmc_args=dict(B_min=2, B_max=2))  # we will force B=2 groups for now

b = state.get_blocks()
print(b.a)

state.draw(pos=g.vp.pos, output_size=(300, 300));

In [ ]:
# What if we don't set the number of groups?

state = minimize_blockmodel_dl(g)
print(state)
state.draw(pos=g.vp.pos, output_size=(300, 300));


In [ ]:
# We can also fit hierarchical SBMs, which have a stronger explanatory power:

state = minimize_nested_blockmodel_dl(g)
state.draw(output_size=(300, 300));

### Exercise  8

* Generate a random graph with 1000 nodes and with a Poisson degree distribution with mean 1.2.
* Draw the graph.
* Fit a SBM (non-nested) on it. How many groups does it find? Do you find this reasonable?

### Exercise 9

* The network `collection.data["dolphins"]` contains the network of friendships between 62 dolphins.
* Use the SBM to investigate its structure: Fit the SBM and visualize the result.